# Krylov Subspace Approximation XL-BOMD (KSA-XL-BOMD)
### see https://pubs.acs.org/doi/full/10.1021/acs.jctc.0c00264 for details.

In [1]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import KSA_XL_BOMD

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [2]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                   }


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set max_rank for a low-rank approximation. See https://pubs.acs.org/doi/full/10.1021/acs.jctc.0c00264 for details.
# Set err_threshold for low-rank approximation error tollerance. Set err_threshold=0.0 to keep max_rank constant.
# T_el - electronic temperature for thermmal HF.
# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.

output = {
'molid': [0,1],
'prefix': 'Outputs/5_KSA_XL-BOMD',
'print_every': 1,
# write a .xyz file
"xyz": {
    "every": 1    # write .xyz file every N steps; 0 disables
    },
# write an HDF5 file
"h5": {
    "data_every": 1,      # write T/Ek/Ep, excitations, MO, etc.; 0 disables
    "dynamics_vectors_every": 1,   # write vel/forces/coords; 0 disables
    "write_mo": False,
    },
}
xl_bomd_params={'k':6, 'max_rank':3, 'err_threshold':0.0, 'T_el':1500}

md = KSA_XL_BOMD(xl_bomd_params=xl_bomd_params, Temp = 400.0,
                   seqm_parameters=seqm_parameters, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 10, remove_com=('angular',1))

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   403.00   1.041830e-01 -4.755826e+02 -4.754784e+02 ||    450.02   1.163387e-01 -1.091493e+02 -1.090329e+02 || 
     2   384.43   9.938378e-02 -4.755778e+02 -4.754784e+02 ||    551.44   1.425591e-01 -1.091757e+02 -1.090332e+02 || 
     3   347.72   8.989228e-02 -4.755682e+02 -4.754783e+02 ||    689.19   1.781688e-01 -1.092117e+02 -1.090336e+02 || 
     4   298.52   7.717366e-02 -4.755553e+02 -4.754782e+02 ||    841.38   2.175141e-01 -1.092515e+02 -1.090340e+02 || 
     5   243.75   6.301442e-02 -4.755411e+02 -4.754781e+02 ||    982.13   2.539010e-01 -1.092884e+02 -1.090344e+02 || 
     6   190.40   4.922176e-02 -4.755272e+02 -4.754780e+02 ||   1085.44   2.806075e-01 -1.093154e+02 -1.090348e+02 || 
     7   144.43   3.733759e-02 -4.755152e+02 -4.754779e+02 ||   1129.98   2.921228e-01 -1.093272e+02 -1.090351e+02 || 
     8   109.93   2.842019e-02 -4.755063e+02 -4.754779e+02 ||   1104.04   2.854176e-01 -1.093205e+02 -1.090351e+

## Damping (Langevin)

In [3]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   }


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set max_rank for a low-rank approximation. See https://pubs.acs.org/doi/full/10.1021/acs.jctc.0c00264 for details.
# Set err_threshold for low-rank approximation error tollerance. Set err_threshold=0.0 to keep max_rank constant.
# T_el - electronic temperature for thermmal HF.
# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.

output = {
'molid': [0,1],
'prefix': 'Outputs/5_KSA_XL-BOMD_damp',
'print_every': 1,
# write a .xyz file
"xyz": {
    "every": 1    # write .xyz file every N steps; 0 disables
    },
# write an HDF5 file
"h5": {
    "data_every": 1,      # write T/Ek/Ep, excitations, MO, etc.; 0 disables
    "dynamics_vectors_every": 1,   # write vel/forces/coords; 0 disables
    "write_mo": False,
    },
}
xl_bomd_params={'k':6, 'max_rank':3, 'err_threshold':0.0, 'T_el':1500}

md = KSA_XL_BOMD(xl_bomd_params=xl_bomd_params, damp=50.0,
                   seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 10, remove_com=('angular',1))

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   400.71   2.071846e-01 -4.755815e+02 -4.753743e+02 ||    436.12   2.254929e-01 -1.091469e+02 -1.089214e+02 || 
     2   387.56   2.003825e-01 -4.755698e+02 -4.753695e+02 ||    458.36   2.369917e-01 -1.091659e+02 -1.089289e+02 || 
     3   332.14   1.717313e-01 -4.755487e+02 -4.753770e+02 ||    449.90   2.326164e-01 -1.091898e+02 -1.089571e+02 || 
     4   268.06   1.385967e-01 -4.755209e+02 -4.753823e+02 ||    476.44   2.463403e-01 -1.092149e+02 -1.089686e+02 || 
     5   258.18   1.334898e-01 -4.754910e+02 -4.753575e+02 ||    532.58   2.753639e-01 -1.092396e+02 -1.089642e+02 || 
     6   199.76   1.032834e-01 -4.754599e+02 -4.753566e+02 ||    563.69   2.914504e-01 -1.092574e+02 -1.089660e+02 || 
     7   141.33   7.307302e-02 -4.754327e+02 -4.753597e+02 ||    604.17   3.123784e-01 -1.092619e+02 -1.089495e+02 || 
     8   121.69   6.291963e-02 -4.754099e+02 -4.753470e+02 ||    661.85   3.422012e-01 -1.092471e+02 -1.089049e+